![Gambar Avengers](Avengers.jpg)
# Collaborative Filtering Movies
###  Oleh :Topik Zulkarnain

Notebook ini bertujuan untuk melakukan suatu rekomendasi dengan menggunakan collaborative filtering terhadap data hasil kuisioner peserta ASTRA Data Scientist Bootcamp. Data kuisioner berupa rating film yang diberikan oleh para peserta data scientist dan dari data tersebut akan dijadikan bahan rekomendasi film apa yang harus direkomendasikan berdasarkan rating yang telah diberikan.
***

# Preparation
## Menyiapkan Data

In [1]:
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
film_data = pd.read_csv('movies.csv')
rating_data = pd.read_csv('rating.csv')

In [3]:
film_data

,MovieID,Title,Genre
0,1,Ada Apa dengan Cinta 2,Drama
1,2,Gundala,Action
2,3,Dilan 1991,Drama
3,4,Bumi Manusia,Drama
4,5,Dua Garis Biru,Drama
5,6,Avengers: End Game,Action
6,7,The Lion King,Animasi
7,8,Aladdin,Animasi
8,9,Spiderman: Far From Home,Action
9,10,Captain Marvel,Action


In [4]:
rating_data.head()

,Nama,MovieID,Rating
0,Hania,1,3
1,Hania,2,5
2,Hania,3,4
3,Hania,4,4
4,Hania,5,4


data yang digunakan adalah data film yang berisi feature MovieID dan title, sedangkan data rating berisi nama pemberi rating, movie id dan juga rating yang diberikan pada tiap film yang pernah ditonton. Berdasarkan data inilah akan dibuat suatu system rekomendasi film untuk seseorang berdasarkan kesamaannya dalam memberikan rating dengan menggunakan collaborative filtering. Pada Collaborative filtering tidak diperlukan data konten pada kasus ini adalah genre. Maka data genre akan didrop.

In [5]:
film_data = film_data.drop(columns=['Genre'])

In [6]:
film_data.head()

,MovieID,Title
0,1,Ada Apa dengan Cinta 2
1,2,Gundala
2,3,Dilan 1991
3,4,Bumi Manusia
4,5,Dua Garis Biru


data siap untuk digunakan

## Collaborative Filtering Movies
Proses pembuatan sistem rekomendasi film adalah sebagai berikut:
- Memilih nama orang yang sama sama sudah menonton film.
- berdasarkan rating yang diberikan, akan dicari tetangga yang paling sering menonton film.
- Menggambil record dari orang yang sudah menonton film.
- Menghitung skor kemiripan.
- Rekomendasikan film berdasarkan skor kemiripan yang tertinggi.

kemudian, misalkan ada seorang yang bernama Fadli, maka dapat direkomendasikan kepada Fadli film apa saja yang dia tonton berdasarkan kemiripan pemberian rating Fadli terhadap data yang ada.

In [7]:
Fadli_Input = [
            {'Title':'Dilan 1991', 'Rating':5},
            {'Title':'Dua Garis Biru', 'Rating':3},
            {'Title':'Captain Marvel', 'Rating':4},
            {'Title':"Spiderman: Far From Home", 'Rating':4},
         ] 
inputMovies = pd.DataFrame(Fadli_Input)
inputMovies

,Rating,Title
0,5,Dilan 1991
1,3,Dua Garis Biru
2,4,Captain Marvel
3,4,Spiderman: Far From Home


Data Fadli harus terlebih dahulu dimasukan feature MovieID agar nanti bisa diproses untuk mendapatkan rekomendasi film

In [8]:
inputId = film_data[film_data['Title'].isin(inputMovies['Title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)
inputMovies

,MovieID,Title,Rating
0,3,Dilan 1991,5
1,5,Dua Garis Biru,3
2,9,Spiderman: Far From Home,4
3,10,Captain Marvel,4


karna sekarang telah diinput movie id, sekarang dapat dicari orang yang telah menonton dan memberikan film yang diinput oleh Fadli

In [9]:
userSubset = rating_data[rating_data['MovieID'].isin(inputMovies['MovieID'].tolist())]
userSubset.head()

,Nama,MovieID,Rating
2,Hania,3,4
4,Hania,5,4
7,Topik Zulkarnain,9,4
8,Topik Zulkarnain,10,2
10,AhokTemanFirli,10,4


data diatas adalah data nama orang yang telah menonton dan memberikan rating film yang diinput Fadli

In [10]:
userSubsetGroup = userSubset.groupby(['Nama'])

Sekarang akan di urutkan group berdasarkan nama orang yang paling sering nonton film berdasarkan input akan lebih diprioritaskan untuk dijadikan rekomendasi

In [11]:
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [12]:
userSubsetGroup[0:3]

[('Jawaharal',           Nama  MovieID  Rating
  94   Jawaharal        3       3
  95   Jawaharal        5       5
  99   Jawaharal        9       5
  100  Jawaharal       10       4), ('OM INDRA',         Nama  MovieID  Rating
  22  OM INDRA        3       2
  23  OM INDRA        5       5
  26  OM INDRA        9       5
  27  OM INDRA       10       5), ('Putrisqiana',
             Nama  MovieID  Rating
  53  Putrisqiana        3       2
  55  Putrisqiana        5       3
  57  Putrisqiana        9       4
  58  Putrisqiana       10       3)]

In [13]:
userSubsetGroup = userSubsetGroup[0:5]

## Menghitung skor kemiripan
Sekarang akan dihitung skor kemiripan antara Fadli dengan data subset orang yang diprioritaskan akan memberikan rekomendasi.

In [14]:
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='MovieID')
    inputMovies = inputMovies.sort_values(by='MovieID')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['MovieID'].isin(group['MovieID'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['Rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['Rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0


In [15]:
pearsonCorrelationDict.items()

dict_items([('Jawaharal', -0.8528028654224417), ('OM INDRA', -0.816496580927726), ('Putrisqiana', -0.5), ('Rima', 0.8164965809277261), ('franadek', 0.8164965809277261)])

In [16]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['Nama'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,Nama
0,-0.852803,Jawaharal
1,-0.816497,OM INDRA
2,-0.500000,Putrisqiana
3,0.816497,Rima
4,0.816497,franadek


#### Rekomendasi dari kemiripan tertinggi
Dapat dilihat dari hasil analisis bahwa Rima dan franadek memberikan skor kemiripan yang paling tinggi. Sealnjutnya ranking akan dibobot lalu akan didaapt rekomendasi film yang harus ditonton oleh Fadli

In [17]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)
topUsers.head()

,similarityIndex,Nama
3,0.816497,Rima
4,0.816497,franadek
2,-0.500000,Putrisqiana
1,-0.816497,OM INDRA
0,-0.852803,Jawaharal


In [18]:
topUsersRating=topUsers.merge(rating_data, left_on='Nama', right_on='Nama', how='inner')
topUsersRating.head()

,similarityIndex,Nama,MovieID,Rating
0,0.816497,Rima,1,5
1,0.816497,Rima,2,5
2,0.816497,Rima,3,5
3,0.816497,Rima,4,4
4,0.816497,Rima,5,3


In [19]:
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Rating']
topUsersRating.head()

,similarityIndex,Nama,MovieID,Rating,weightedRating
0,0.816497,Rima,1,5,4.082483
1,0.816497,Rima,2,5,4.082483
2,0.816497,Rima,3,5,4.082483
3,0.816497,Rima,4,4,3.265986
4,0.816497,Rima,5,3,2.449490


In [20]:
tempTopUsersRating = topUsersRating.groupby('MovieID').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
MovieID,,
1,-0.536306,1.193374
2,0.280190,2.437258
3,-0.536306,2.157067
4,1.132993,5.348469
5,-0.536306,-4.947518


In [21]:
recommendation_df = pd.DataFrame()
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
MovieID,,
1,-2.225172,1
2,8.698580,2
3,-4.022081,3
4,4.720654,4
5,9.225172,5


In [22]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head()

,weighted average recommendation score,movieId
MovieID,,
5,9.225172,5
2,8.698580,2
9,5.590141,9
6,5.000000,6
4,4.720654,4


In [23]:
film_data.loc[film_data['MovieID'].isin(recommendation_df.head(5)['movieId'].tolist())]

,MovieID,Title
1,2,Gundala
3,4,Bumi Manusia
4,5,Dua Garis Biru
5,6,Avengers: End Game
8,9,Spiderman: Far From Home


## Kesimpulan

Film yang direkomendasikan untuk Fadli adalah Bumi Manusia, Avengers: Endgame, gundala